# Population-based Heuristics
## Solving Knapsack problem

### Imports and Declaration

In [1]:
import random
import copy
import math
import queue
import numpy as np

In [2]:
def generateVals(length, vmax, wmax):
    gbValues = []
    gbWeights = []

    while len(gbValues) < length:
        gbValues.append(random.randint(1,vmax))
        gbWeights.append(random.randint(1,wmax))
        
    return gbValues, gbWeights

In [3]:
'''
Declared global variables for the knapsack problem
'''
gValues, gWeights = generateVals(50, 75, 50)
fullWeight = 500

In [4]:
print("List of values of items: ", gValues)
print("List of weights of items: ", gWeights)

List of values of items:  [9, 16, 28, 36, 20, 60, 65, 60, 1, 70, 13, 47, 3, 72, 31, 17, 49, 25, 51, 19, 4, 24, 31, 74, 38, 15, 35, 29, 31, 39, 57, 1, 11, 8, 3, 31, 3, 7, 69, 55, 22, 38, 69, 48, 14, 39, 75, 2, 10, 52]
List of weights of items:  [6, 10, 48, 43, 30, 48, 38, 15, 38, 7, 29, 43, 47, 42, 21, 30, 16, 48, 21, 15, 32, 32, 45, 5, 12, 49, 14, 5, 44, 44, 13, 18, 22, 15, 22, 12, 31, 9, 24, 8, 10, 7, 45, 6, 30, 29, 9, 25, 47, 42]


## Generic functions used in different methods

In [5]:
'''
Create random solution
input: s <-- size of the list
output: [0,1,1,0,0,1]
'''
def randomSolution(s):
    temp = []
    for i in range(s):
        temp.append(random.randrange(2))
        
    return temp

In [6]:
'''
Retrive the total weight and total value of the provided solution
input: solution <-- bit-list solution, identifies which items to include in the knapsack
output: total weight, total value
'''
def getWV(solu):
    tWeight = 0
    tValue = 0
    for i in range(len(solu)):
        if solu[i] == 1:
            tWeight += gWeights[i]
            tValue += gValues[i]
            
    return tWeight, tValue

In [7]:
'''
Function to determine if the solution fits the constraint for the knapsack problem
input: solution <-- bit-list solution, identifies which items to include in the knapsack
output: 0 or 1
'''
def fitnessW(solu):
    tempW, tempV = getWV(solu)
    if tempW > fullWeight:
        return 0
    return 1

In [8]:
'''
Function to flip the last bit of the solution
input: solution <-- bit-list solution, identifies which items to include in the knapsack
output: solution
'''
def tweak(sol):
    carry = False
    for i in reversed(range(len(sol))):
        if carry:
            if sol[i] > 0:
                sol[i] = 0
                carry = True
            else:
                sol[i] = 1
                carry = False
        else:
            if sol[i] > 0:
                sol[i] = 0
                carry = True
            else:
                sol[i] = 1
                carry = False
                break
                
    return sol

### Genetic Algorithm

#### _Fitness Function_

In [9]:
'''
No exact fitness calculated but on the comparison of the two solutions.
SolutionY is made sure to have passed the constraint for the weight
before it can be included in the function
input: solutionX, and solutionY <-- bit-list solution, identifies which items to include in the knapsack
output: solutionX or solutionY
'''
def fitnessGA(solX, solY):
    if solX.fitness:
        return False
        
    else:
        if solX.ks_value > solY.ks_value:
            return True
        else:
            return False

In [10]:
'''
Create a class solution which contains the necessary information for a solution,
i.e. fitness, total weight and value, print function.
'''
class Solution:
    def getWV(self, solu, values, weights):
        tWeight = 0
        tValue = 0
        for i in range(len(solu)):
            if solu[i] == 1:
                tWeight += weights[i]
                tValue += values[i]

        return tWeight, tValue
    
    def __init__(self, sol, vals, wts):
        self.solution = sol
        self.values = vals
        self.weights = wts
        self.ks_weight, self.ks_value = self.getWV(sol, self.values, self.weights)
        self.fitness = self.ks_weight > fullWeight
        
    def show(self):
        for i in range(len(self.values) - 1):
            if self.solution[i] == 1:
                print("Item: {}, Value: {}, Weight: {}".format(i,self.values[i], self.weights[i]))

        print("Total Value and Weight: {}, {}".format(self.ks_value, self.ks_weight))
        print("=========")
        

In [11]:
'''
This function compares the different solutions inside the population.
It is sorted by each solution's knapsack total value
input: population
output: sorted population based on knapsack value
'''
def solComp(pop):
    return sorted(pop, key=lambda x: x.ks_value, reverse=True)

In [12]:
'''
Function for selecting parents for breeding.
Only the bottom 80% of the population will be considered for breeding.
input: sorted population
output: index of solution in the population
'''
def selectParent(pop):
    size = int(len(pop) * 0.80)
    rIdx = random.randrange(size)
    return pop[rIdx]

In [13]:
'''
Function for breeding new children (solution) with crossover point
at the middle section.
input: parent 1, parent 2 (both are objects of solution class)
output: child 1, child 2 (both are objects of solution class)
'''
def crossover(p1, p2):
    cTemp1 = []
    cTemp2 = []
    crossPoint = int(len(p1.solution) / 2)
    
    cTemp1 = p1.solution[:crossPoint] + p2.solution[crossPoint:]
    cTemp2 = p2.solution[:crossPoint] + p1.solution[crossPoint:]
    
    c1 = Solution(cTemp1, gValues, gWeights)
    c2 = Solution(cTemp2, gValues, gWeights)
    
    return c1, c2

In [14]:
'''
Function to mutate child. This is a simple bit flip.
The probability of the child mutating is found within the geneticAlgo function
input: child (Solution object)
output: mutated child (new Solution object)
'''
def mutate(child):
    temp = []
    for i in child.solution:
        if random.randrange(2) == 1:
            if i == 0:
                i = 1
            else:
                i = 0
        temp.append(i)
        
    return Solution(temp, gValues, gWeights)

In [15]:
def geneticAlgo():
    solSize = len(gValues)
    popSize = 30
    topPoint = int(popSize * 0.8)
    cutOff = int(popSize * 0.2)
    popList = []
    generation = 25
    
    for i in range(popSize):
        popList.append(Solution(randomSolution(solSize), gValues, gWeights))
        
    sGA = popList[random.randrange(popSize-1)]
    while sGA.fitness == True:
        sGA = popList[random.randrange(popSize-1)]
    
    bestGA = copy.deepcopy(sGA)
    # start checking per generation
    for i in reversed(range(generation)):
        for p in popList:
            if fitnessGA(p, bestGA):                   # (bestGA.solution == sGA.solution) since bestGA is already set to be within fitness, no need to remove it
                bestGA = copy.deepcopy(p)
    
        qList = []
        
        # sort population based on their values
        popList = solComp(popList)
        
        for i in popList[topPoint:]:
            if not i.fitness:
                qList.append(i)
                
        for i in popList[cutOff:topPoint]:
            qList.append(i)
            
        # breeding
        while len(qList) < popSize:
            parent1 = selectParent(popList)
            parent2 = selectParent(popList)
            while parent1.solution == parent2.solution:
                parent2 = selectParent(popList)
                
            child1, child2 = crossover(parent1, parent2)
            
            # randomized upperbound for chance of a child mutating
            if random.uniform(0.0, 1.0) > 0.69:
                child1 = mutate(child1)
                
            # second child having higher chances of mutating
            if random.uniform(0.0, 1.0) > 0.75:
                child2 = mutate(child2)
                
            qList.append(child1)
            qList.append(child2)
            
        popList = copy.deepcopy(qList)
        
    bestGA.show()

In [16]:
%%time
for i in range(5):
    geneticAlgo()

Item: 0, Value: 9, Weight: 6
Item: 1, Value: 16, Weight: 10
Item: 2, Value: 28, Weight: 48
Item: 4, Value: 20, Weight: 30
Item: 7, Value: 60, Weight: 15
Item: 10, Value: 13, Weight: 29
Item: 11, Value: 47, Weight: 43
Item: 13, Value: 72, Weight: 42
Item: 15, Value: 17, Weight: 30
Item: 16, Value: 49, Weight: 16
Item: 17, Value: 25, Weight: 48
Item: 23, Value: 74, Weight: 5
Item: 24, Value: 38, Weight: 12
Item: 25, Value: 15, Weight: 49
Item: 27, Value: 29, Weight: 5
Item: 33, Value: 8, Weight: 15
Item: 36, Value: 3, Weight: 31
Item: 40, Value: 22, Weight: 10
Item: 42, Value: 69, Weight: 45
Item: 46, Value: 75, Weight: 9
Total Value and Weight: 689, 498
Item: 1, Value: 16, Weight: 10
Item: 3, Value: 36, Weight: 43
Item: 6, Value: 65, Weight: 38
Item: 7, Value: 60, Weight: 15
Item: 11, Value: 47, Weight: 43
Item: 13, Value: 72, Weight: 42
Item: 14, Value: 31, Weight: 21
Item: 15, Value: 17, Weight: 30
Item: 18, Value: 51, Weight: 21
Item: 23, Value: 74, Weight: 5
Item: 27, Value: 29, Wei

## Ant Colony Optimisation

In [23]:
'''
Function for updating pheromone levels of each component.
Simplistic approach with desirability by adding 1 whenever the component is used.
Similar with evaporation, cuts the pheromone level by half.
input: pheromone trail
        population/list of different trails
output: updated pheromone trail
'''
def updatePheromone(phero, pop):
    # desirability: increase components desirability by 1 when used in solution
    for i in range(len(phero)-1):
        for j in pop:
            if j.solution[i] == 1:
                phero[i] += 1
    
    # evaporate: cuts the pheromone level by half after every generation
    for i in phero:
        i = int(i/4)
        
    return phero

In [37]:
'''
Building solution or trail. Initial trail building is randomized
since there is no associated pheromone for each component yet.
input: pheromone trail
        size of the trail
output: solution trail (Solution object)
'''
def trailBuild(phero, size):
    temp = list(np.zeros(size, dtype=int))
    rankPhero = np.argsort(phero)
    uBound = 0.95
    chances = 0.05
    prev = 0
    if all(x == 0 for x in phero):
        return Solution(randomSolution(size), gValues, gWeights)
    
    for i in reversed(rankPhero):
        if fitnessW(temp) == 0:
            temp[prev] = 0
            return Solution(tweak(temp), gValues, gWeights)                   # added tweaking to trail found based on the pheromone
        else:
            if random.uniform(0.0,1.0) > uBound:                           # added likelihood of the component being chosen
                temp[i] = 1
                prev = copy.deepcopy(i)
                uBound -= chances
            else:
                temp[i] = 0
                prev = copy.deepcopy(i)
                uBound -= chances

In [19]:
'''
No exact fitness calculated but on the comparison of the two solutions.
SolutionY is made sure to have passed the constraint for the weight
before it can be included in the function
input: solutionX, and solutionY <-- bit-list solution, identifies which items to include in the knapsack
output: solutionX or solutionY
'''
def fitnessACO(solX, solY):
    if solX.fitness:
        return False
        
    else:
        if solX.ks_value > solY.ks_value:
            return True
        else:
            return False

In [35]:
def antColony():
    trailSize = len(data.iloc[0,:])
    pheroTrail = list(np.zeros(trailSize, dtype=int))
    countdown = 50
    stagnant = 5
    trailExplore = 20
    popTrail = []
        
    sACO = solution(randomSolution(trailSize))
    sACO.updateFitness()
        
    bestACO = copy.deepcopy(sACO)
        
    while countdown > 0 or stagnant > 0:
        for t in range(trailExplore):
            popTrail.append(trailBuild(pheroTrail, trailSize))
        
        for i in popTrail:
            i.updateFitness()
            if fitnessACO(i, bestACO):          #bestACO.solution == sACO.solution
                bestACO = copy.deepcopy(i)
            else:
                stagnant -= 1
        
        pheroTrail = updatePheromone(pheroTrail, popTrail)
        countdown -= 1
        
    bestACO.show()

In [38]:
%%time
for i in range(5):
    antColony()

Item: 1, Value: 16, Weight: 10
Item: 2, Value: 28, Weight: 48
Item: 5, Value: 60, Weight: 48
Item: 6, Value: 65, Weight: 38
Item: 7, Value: 60, Weight: 15
Item: 9, Value: 70, Weight: 7
Item: 11, Value: 47, Weight: 43
Item: 12, Value: 3, Weight: 47
Item: 19, Value: 19, Weight: 15
Item: 21, Value: 24, Weight: 32
Item: 23, Value: 74, Weight: 5
Item: 24, Value: 38, Weight: 12
Item: 28, Value: 31, Weight: 44
Item: 30, Value: 57, Weight: 13
Item: 38, Value: 69, Weight: 24
Item: 40, Value: 22, Weight: 10
Item: 43, Value: 48, Weight: 6
Item: 44, Value: 14, Weight: 30
Item: 46, Value: 75, Weight: 9
Total Value and Weight: 872, 498
Item: 5, Value: 60, Weight: 48
Item: 6, Value: 65, Weight: 38
Item: 7, Value: 60, Weight: 15
Item: 11, Value: 47, Weight: 43
Item: 13, Value: 72, Weight: 42
Item: 14, Value: 31, Weight: 21
Item: 15, Value: 17, Weight: 30
Item: 16, Value: 49, Weight: 16
Item: 18, Value: 51, Weight: 21
Item: 19, Value: 19, Weight: 15
Item: 23, Value: 74, Weight: 5
Item: 24, Value: 38, W